## this notebook is for interactive vizualisation of a given gradcam heatmap 

In [1]:
from ipywidgets import widgets
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import nibabel as nib
from skimage.transform import resize


In [2]:
MNI_1mm = nib.load('MNI152_T1_1mm_brain.nii.gz').get_fdata()
MNI_2mm = nib.load('MNI152_T1_2mm_brain.nii.gz').get_fdata()
MNI_1mm = resize(MNI_1mm,(120,140,120))

In [3]:
hpath_1mm='gcampadded1mm.npy'
hpath_2mm='gcampadded2mm.npy'
#dpath='/home/shane/datasets/cropped_local_data.npy'
hmap_1mm = np.load(hpath_1mm)
hmap_2mm = np.load(hpath_2mm)
#data_arr = np.load(dpath)

In [4]:
def get_average_hmap(hmap):
    valid_mask = [np.any(~np.isnan(hmap[i])) for i in range(hmap.shape[0])]
    valid_entries = hmap[valid_mask]
    return np.average(valid_entries,axis=0)

hmap_1mm_av=get_average_hmap(hmap_1mm)
hmap_2mm_av=get_average_hmap(hmap_2mm)

In [5]:
# create mask for viz 
def create_mask(data,hmap):
    msk = data > 0.0
    msk = msk*1
    return hmap * msk

masked_hmap_1mm = create_mask(MNI_1mm,hmap_1mm_av)
masked_hmap_2mm = create_mask(MNI_2mm,hmap_2mm_av)
        

In [6]:
def create_gui(data,hmap_,title):
   # data=MNI_1mm# default patient to vizualise
    #hmap_=hmap[pat]
   # hmap_=masked_hmap
    def f(ind,plane,alpha):
        fig = plt.figure(figsize=(15,8))
        if plane == 'Saggital':
            im1 = plt.imshow(np.rot90(data[ind]),cmap='gray')
            im2 = plt.imshow(np.rot90(hmap_[ind]), cmap='jet', alpha=alpha)
        if plane == 'Coronal':
            im1 = plt.imshow(np.rot90(data[:,ind]),cmap='gray')
            im2 = plt.imshow(np.rot90(hmap_[:,ind]), cmap='jet', alpha=alpha)
        if plane == 'Axial':
            im1 = plt.imshow(np.rot90(data[:,:,ind]),cmap='gray')
            im2 = plt.imshow(np.rot90(hmap_[:,:,ind]), cmap='jet', alpha=alpha)
        fig.colorbar(im2)
        fig.suptitle(title)
        _=plt.show()

    from ipywidgets import widgets

    ind = widgets.IntSlider(min=0,max=data.shape[0],value=50)
    plane = widgets.Dropdown(options=['Saggital','Coronal','Axial'],value='Coronal')
    alpha = widgets.FloatSlider(min=0.0,max=1.0,value=0.4,step=0.01)
    ui = widgets.VBox([widgets.Label("Plane:"),plane,widgets.Label("Index:"),ind, widgets.Label("Alpha:"),alpha])

    out = widgets.interactive_output(f, {'ind': ind, 'plane': plane, 'alpha': alpha})
    return ui, out
ui_1mm, out_1mm = create_gui(MNI_1mm,masked_hmap_1mm,'1mm subsample with padding')
ui_2mm, out_2mm = create_gui(MNI_2mm,masked_hmap_2mm,'2mm full with padding')



In [7]:
widgets.HBox([ui_1mm,out_1mm])

In [8]:
widgets.HBox([ui_2mm,out_2mm])